In [3]:
from dataclasses import dataclass

import pickle

import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions

from bs4 import BeautifulSoup

In [4]:
options = ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

In [96]:
url='https://www.hl.co.uk/shares/shares-search-results/3216473' # JEMA
# url='https://www.hl.co.uk/shares/shares-search-results/B1YW440' #iii

In [127]:
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html')

In [107]:
soup

<html class="js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" lang="en"><!--  <![endif]--><head>
<meta charset="utf-8"/>
<title>3i Group Plc Share Price (III) Ordinary 73 19/22p | III</title>
<meta content="The latest 3i Group Plc (III) Ordinary 73 19/22p share price (III). View recent trades and share price information for 3i Group Plc (III) Ordinary 73 19/22p" name="Description"/>
<meta content="3i, Group, Ordinary, 73, 19/22p,III,share,stock,holding,price,latest,trading,volume,change,epic" name="Keywords"/>
<meta content="3i Group Plc (III) Ordinary 73 19/22p share price 

In [8]:
soup.find_all(text=re.compile('Premium/Discount'))

/tmp/ipykernel_1804626/692484421.py:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  soup.find_all(text=re.compile('Premium/Discount'))


['Premium/Discount',
 'Premium/Discount',
 'Premium/Discount',
 '12m average Premium/Discount',
 "The average Premium/Discount between the trust's share price and Net Asset Value over the last                12 months.            "]

In [9]:
spans = soup.find_all('span')
spans

[<span class="topNav__insightHide"> &amp; Insight</span>,
 <span class="divider">»</span>,
 <span class="divider">»</span>,
 <span class="divider">»</span>,
 <span>
             ORD GBP0.01
                     </span>,
 <span class="price-label">Sell:</span>,
 <span class="bid price-divide" data-field="bid" data-grid="stocks" data-item="JEMA-L" data-source="lightstreamer" id="ls-bid-JEMA-L">132.60p</span>,
 <span class="price-label">Buy:</span>,
 <span class="ask price-divide" data-field="ask" data-grid="stocks" data-item="JEMA-L" data-source="lightstreamer" id="ls-ask-JEMA-L">141.40p</span>,
 <span class="dNone" data-field="mid" data-grid="stocks" data-item="JEMA-L" data-source="lightstreamer" id="ls-mid-JEMA-L">0</span>,
 <span class="price-label large-hide medium-hide">Change:</span>,
 <span class="change-divide">
 <span class="change-arrow">
 <img id="stock_change_arrow" src="https://online.hl.co.uk/img/hl/layout/security-down-arrow.gif"/>
 </span>
 <span class="negative change" d

In [10]:
matching_spans = []

for span in spans:
    res = span.find_all(string=re.compile('Premium/Discount'))
    if (len(res)):
        print(res)
        matching_spans.append(res)

['Premium/Discount']
['Premium/Discount']
['12m average Premium/Discount']
["The average Premium/Discount between the trust's share price and Net Asset Value over the last                12 months.            "]


In [11]:
sec_det = soup.find(id='security-detail')
display(sec_det)

<div id="security-detail">
<div class="row">
<div class="columns large-3 medium-4 small-6">
<span aria-describedby="tooltip-lnacod7m3" aria-haspopup="true" class="has-tip" data-selector="tooltip-lnacod7m3" data-tooltip="" title="">Open</span>:<br class="large-hide wide-medium-hide"/>
<strong>135.80p</strong>
</div>
<div class="columns large-3 medium-4 small-6">
<span aria-describedby="tooltip-lnacod7m4" aria-haspopup="true" class="has-tip" data-selector="tooltip-lnacod7m4" data-tooltip="" title="">Trade high</span>:<br class="large-hide wide-medium-hide"/>
<strong>138.71p</strong>
</div>
<div class="columns large-3 medium-4 small-6">
<span aria-describedby="tooltip-lnacod7m5" aria-haspopup="true" class="has-tip" data-selector="tooltip-lnacod7m5" data-tooltip="" title="">Year high</span>:<br class="large-hide wide-medium-hide"/>
<strong>147.20p</strong>
</div>
<div class="columns large-3 medium-4 small-6">
<span aria-describedby="tooltip-lnacod7m6" aria-haspopup="true" class="has-tip" d

In [36]:
premium = sec_det.find_all('strong')[9].text
premium

'n/a'

In [13]:
float(premium.strip('%'))

201.24

In [14]:
def get_nav_premium(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html')
    sec_det = soup.find(id='security-detail')
    premium = sec_det.find_all('strong')[7].text
    return float(premium.strip('%'))

get_nav_premium(url)

201.24

In [128]:
idx_nav_premium = 7
idx_nav = 3
idx_volume = 8
idx_div_yld = 9

annual_mgmt_charge_idx = 1
performance_fee_idx = 2
ongoing_charge_idx = 3
total_assets_idx = 6
gross_gearing_idx = 7
market_cap_idx = 8
structure_idx = 10
domicile_idx = 11
isin_idx = 12


def get_sec_detail(soup, idx):
    sec_det = soup.find(id='security-detail')
    return sec_det.find_all('strong')[idx].text

def get_factsheet_detail(soup, idx):
    factsheet_table = soup.find(class_='factsheet-table')
    return factsheet_table.find_all(class_='align-left')[idx].text.replace('\n', '').replace('\t', '').replace(' ', '')

def get_trust_basics(soup, idx):
    trust_basics = soup.find_all(class_='factsheet-table table-no-border spacer-bottom')[0]
    return trust_basics.find_all('td')[idx].text.replace('\n', '').replace('\t', '')

def get_objective(soup):
    return soup.find_all("div", class_="grey-gradient clearfix")[1].text.replace('\n', '').replace('\t', '')

def get_symbol_data(soup):    

    nav = get_sec_detail_float(soup, idx_nav)
    nav_premium = get_sec_detail_float(soup, idx_nav_premium)
    volume = get_sec_detail_float(soup, idx_volume)
    div_yld = get_sec_detail_float(soup, idx_div_yld)

    annual_mgmt_charge = get_trust_basics(soup, annual_mgmt_charge_idx).replace(' ', '')
    performance_fee = get_trust_basics(soup, performance_fee_idx).replace(' ', '')
    ongoing_charge = get_trust_basics(soup, ongoing_charge_idx).replace(' ', '')
    total_assets = get_trust_basics(soup, total_assets_idx).replace(' ', '')
    gross_gearing = get_trust_basics(soup, gross_gearing_idx).replace(' ', '')
    market_cap = get_trust_basics(soup, market_cap_idx).replace(' ', '')
    structure = get_trust_basics(soup, structure_idx)
    domicile = get_trust_basics(soup, domicile_idx)
    isin = get_trust_basics(soup, isin_idx)

    objective = get_objective(soup)

    return nav, nav_premium, volume, div_yld, annual_mgmt_charge, performance_fee, ongoing_charge, total_assets, gross_gearing, market_cap, structure, domicile, isin, objective
    
get_symbol_data(soup)

('1,825.90',
 '13.53%',
 '855,919',
 '2.65%',
 'n/a',
 'Yes',
 '1.08%',
 '£18,547m',
 '4%',
 '£20,177m',
 'Closed Ended Investment Company',
 'United Kingdom',
 'GB00B1YW4409',
 '3i is an investment company which aims to provide its shareholders with quoted access to private equity and infrastructure returns. Currently, its main focus is on making quoted and unquoted equity and/ or debt investments in businesses and funds in Europe, Asia and the Americas.')

In [123]:
trust_basics = soup.find_all(class_='factsheet-table table-no-border spacer-bottom')[0]
trust_basics.find_all('td')[2].text.replace('\n', '').replace('\t', '').replace(' ', '')

'Yes'